### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import re

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [6]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

#Defining the query
query = "movie reviews"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
api_url = (
    f"{url}q={query}&fq={filter_query}&sort={sort}&fl={field_list}"
    f"&begin_date={begin_date}&end_date={end_date}&api-key={nyt_api_key}"
)

In [10]:
# Create an empty list to store the reviews
reviews = []

# loop through pages 0-19
for i in range(0,20):
    # create query with a page number
    # API results show 10 articles at a time
    i_url = api_url + '&page=' + str(i)
    
    # Make a "GET" request and retrieve the JSON
    res = requests.get(i_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    for j in range(0, len(res["response"]["docs"])):
        try:
            # loop through the reviews["response"]["docs"] and append each review to the list
            reviews.append(res["response"]["docs"][j])
            # Print the page that was just retrieved
        except KeyError:
            # Print the page number that had no results then break from the loop
            print(f"Page {i} had no result")

    print(f"Checked page {i}")

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [12]:
len(reviews)

200

In [14]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews[0:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [16]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
197,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
198,https://www.nytimes.com/2017/02/09/movies/kedi...,This documentary profiles distinctive street c...,The New York Times,"[{'name': 'creative_works', 'value': 'Kedi (Mo...",2017-02-09T16:21:39+0000,274,"Review: Cute Cats of ‘Kedi,’ Rekindling a ‘Lov...",None,None,Kedi,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None


In [18]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
# pattern = r'‘(.*?)’ Review'
    
    # pattern = r'\u2018(.*?)'

    # pattern = r'‘(.*?)’\s*Review|Review:\s*‘(.*?)’'
    
    # pattern = r'\u2018(.*?)\u2019 Review|Review: \u2018(.*?)\u2019 '
    
    # pattern = r"‘(.*?)’(?:\s*Review|(?:\s*Review:))"
    
    # pattern = r'\u2018(.*?)\u2019(?:\s*Review|(?:\s*Review:))'

    # pattern = r'\u2018(.*?)\u2019(?:\s*Review|(?:\s*Review:))'
    
    # match = re.search(pattern, s)
    
    # if match:
    #     # Check which group has the matched title
    #     if match.group(1):
    #         return match.group(1)
    #     elif match.group(2):
    #         return match.group(2)
    # return None

    # if match:
    #     return match.group(1)
    # return None



def extract_title(s):
    f = False #Flag to check whether to append or not
    s1 = "" #Empty string literal to get the title
    for i,c in enumerate(s):
        try:
            if c == '\u2018':
                f = True # Start appending
            elif c == '\u2019' and i != len(s)-1 and  s[i+1] == " ":
                f = False #Stop appending
            elif f and i != len(s)-1:
                s1 += c #Append
        except:
            print('---------------------------------------------')
            print(f"Error thrown at: {s}")
            print('---------------------------------------------')
            pass
    
    if len(s1) == 0: #Empty String returns the input string
        s1 = s
        
    if s1[-1] == ',': #Incase the headline contains invalid end with ","
        s1 = s1[:-1]
        
    return s1




reviews_df['title'] = reviews_df['headline.main'].apply(extract_title)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
196,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
197,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Everybody Loves Somebody
198,https://www.nytimes.com/2017/02/09/movies/kedi...,This documentary profiles distinctive street c...,The New York Times,"[{'name': 'creative_works', 'value': 'Kedi (Mo...",2017-02-09T16:21:39+0000,274,"Review: Cute Cats of ‘Kedi,’ Rekindling a ‘Lov...",None,None,Kedi,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,"Kedi,Love of Life"


In [20]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df_k = reviews_df
reviews_df_k['keywords'] = reviews_df_k['keywords'].apply(extract_keywords)
reviews_df_k

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
196,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
197,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Everybody Loves Somebody
198,https://www.nytimes.com/2017/02/09/movies/kedi...,This documentary profiles distinctive street c...,The New York Times,creative_works: Kedi (Movie);subject: Document...,2017-02-09T16:21:39+0000,274,"Review: Cute Cats of ‘Kedi,’ Rekindling a ‘Lov...",None,None,Kedi,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,"Kedi,Love of Life"


In [22]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
movies_title_list = reviews_df['title'].to_list()
movies_title_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West 

### Access The Movie Database API

In [25]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
url_details = "https://api.themoviedb.org/3/movie/"

In [27]:
xyz = "What’s Love Got to Do With It?"

tempreq = requests.get(f"{url}{xyz}{tmdb_key_string}").json()

# print(json.dumps(tempreq['results'][0],indent=4))
hit_number = 0
if tempreq['total_results'] > 1:
    for x in range(0,len(tempreq['results'])):
        if tempreq['results'][x]['title'].lower() == xyz.lower():
            hit_number = x
            break

if tempreq['total_results'] > 0:
    
    # print(f"Total_results: {tempreq['total_results']}")
    # print(json.dumps(tempreq['results'][hit_number],indent=4))
    # print(tempreq['results'][hit_number]['id'])
    
    id = str(tempreq['results'][hit_number]['id']) + '?'

    # print(url_details+id+tmdb_key_string)
    details = requests.get(url_details+id+tmdb_key_string).json()

    
    # print(json.dumps(details, indent=4))
    
    genre_list = [genre["name"] for genre in details["genres"]]
    # print(genre_list)

    spoken_languages_list = [spoken_languages["name"] for spoken_languages in details["spoken_languages"]]
    # print(spoken_languages_list)

    production_countries_list = [production_countries["name"] for production_countries in details["production_countries"]]
    # print(production_countries_list)
    try:
        details_dictionary = {"title":details["title"],
                              "original_title":details["original_title"],
                              "budget":details["budget"],
                              "genre":genre_list,
                              "language":details["original_language"],
                              "spoken_languages":spoken_languages_list,
                              "homepage":details["homepage"],
                              "overview":details["overview"],
                              "popularity":details["popularity"],
                              "runtime":details["runtime"],
                              "revenue":details["revenue"],
                              "release_date":details["release_date"],
                              "vote_average":details["vote_average"],
                              "vote_count":details["vote_count"],
                              "production_countries":production_countries_list}
    except KeyError:
        print('KeyError')
        pass
        
    print(json.dumps(details_dictionary, indent = 4))

else:
    print('No results')

{
    "title": "What's Love Got to Do with It?",
    "original_title": "What's Love Got to Do with It?",
    "budget": 0,
    "genre": [
        "Romance",
        "Comedy"
    ],
    "language": "en",
    "spoken_languages": [
        "English",
        "Portugu\u00eas",
        "\u0627\u0631\u062f\u0648"
    ],
    "homepage": "",
    "overview": "Two childhood friends now in their thirties must decide whether to follow their heads or their hearts once the man decides to follow his parents' advice and enter into an arranged marriage in Pakistan.",
    "popularity": 15.937,
    "runtime": 109,
    "revenue": 10898395,
    "release_date": "2023-01-26",
    "vote_average": 6.083,
    "vote_count": 180,
    "production_countries": [
        "France",
        "United Kingdom"
    ]
}


In [55]:
# Create an empty list to store the results
tmdb_movies_list = []
hit_number = 0

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0
for title in movies_title_list:
# Loop through the titles

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter > 0:
        time.sleep(1)
        print(f"Sleeping at {request_counter}")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    result = requests.get(url+title+tmdb_key_string).json()


    result = [movie for movie in result['results'] if movie['title'].lower() == title.lower() or movie['original_title'].lower() == title.lower()]

    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = str(result[0]['id']) + '?'
        
        # Make a request for a the full movie details
        movie_details = requests.get(url_details+movie_id+tmdb_key_string).json()

        # Execute "GET" request with url
        
        # Extract the genre names into a list
        genre_list = [genre["name"] for genre in movie_details["genres"]]
    
        # Extract the spoken_languages' English name into a list
        spoken_languages_list = [spoken_languages["name"] for spoken_languages in movie_details["spoken_languages"]]
    
        # Extract the production_countries' name into a list
        production_countries_list = [production_countries["name"] for production_countries in movie_details["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        movie_details_dictionary = {"title":movie_details["title"],
                              "original_title":movie_details["original_title"],
                              "budget":movie_details["budget"],
                              "genre":genre_list,
                              "language":movie_details["original_language"],
                              "spoken_languages":spoken_languages_list,
                              "homepage":movie_details["homepage"],
                              "overview":movie_details["overview"],
                              "popularity":movie_details["popularity"],
                              "runtime":movie_details["runtime"],
                              "revenue":movie_details["revenue"],
                              "release_date":movie_details["release_date"],
                              "vote_average":movie_details["vote_average"],
                              "vote_count":movie_details["vote_count"],
                              "production_countries":production_countries_list}
        tmdb_movies_list.append(movie_details_dictionary)
        
        # Print out the title that was found
        print(f"Found {title}")
    except:
        print(f"{title} not found.")
        

Found The Attachment Diaries
What’s Love Got to Do With It? not found.
Found You Can Live Forever
A Tourist’s Guide to Love not found.
Other People’s Children not found.
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Lady Chatterley’s Lover not found.
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Love & Gelato not found.
Found Stay Prayed Up
Found Benediction
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
Found Cyrano
Found T

In [ ]:
# Create an empty list to store the results

tmdb_movies_list = []
hit_number = 0

# Create a request counter to sleep the requests after a multiple
# of 50 requests

request_counter = 0

# Loop through the titles
for title in movies_title_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter > 0:
        print('- - - - - - - - - - - - - -')
        print(f'Sleeping at {request_counter} Requests')
        print('- - - - - - - - - - - - - -')
        time.sleep(10)
        
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    res = requests.get(url+title+tmdb_key_string).json()
    
    # Include a try clause to search for the full movie details.
    try:
        hit_number = 0
        if tempreq['total_results'] > 1:
            for x in range(0,len(tempreq['results'])):
                if tempreq['results'][x]['title'].lower() == title.lower():
                    hit_number = x
                    print(f"Found {title}.")
                    break
        elif tempreq['total_results'] > 0:
            # print(f"Total_results for {title}: {res['total_results']}")
            # res['results'][0]
            print(f"Found {title}.")
            # print('______________')
        else:
            print(f"---{title} not found.---")
            
    # Use the except clause to print out a statement if a movie is not found.
    except KeyError:
        print(f"---{title} not found.---")


    
    
        # Get movie id
    try:
        id = str(res['results'][hit_number]['id']) + '?'
    except KeyError:
        print('Error during ID retrieval')

    try:
        # Make a request for a the full movie details
        details_req = requests.get(url_details+id+tmdb_key_string)
        details = details_req.json()
    except:
        print(f'Something went wrong: {details_req}')
        continue
    
    # Execute "GET" request with url --- done in previous lines.
    
    try:
        # Extract the genre names into a list
        genre_list = [genre["name"] for genre in details["genres"]]
        
        # Extract the spoken_languages' English name into a list
        spoken_languages_list = [spoken_languages["name"] for spoken_languages in details["spoken_languages"]]
    
        # Extract the production_countries' name into a list
        production_countries_list = [production_countries["name"] for production_countries in details["production_countries"]]
        
    except:
        print('Could not retrieve lists')
        
    # Add the relevant data to a dictionary and
    # append it to the tmdb_movies_list list
    try:
        details_dictionary = {"title":details["title"],
                              "original_title":details["original_title"],
                              "budget":details["budget"],
                              "genre":genre_list,
                              "language":details["original_language"],
                              "spoken_languages":spoken_languages_list,
                              "homepage":details["homepage"],
                              "overview":details["overview"],
                              "popularity":details["popularity"],
                              "runtime":details["runtime"],
                              "revenue":details["revenue"],
                              "release_date":details["release_date"],
                              "vote_average":details["vote_average"],
                              "vote_count":details["vote_count"],
                              "production_countries":production_countries_list}
        tmdb_movies_list.append(details_dictionary)
        
    except KeyError:
        print('KeyError') 


        # Print out the title that was found
        ## Done Previously


In [31]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[0:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Espa\u00f1ol"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.278,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "production_countries": [
            "Argentina"
        ]
    },

In [33]:
# Convert the results to a DataFrame
tmdb_movies_list_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_list_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Español],,"Argentina, 1970s. A desperate young woman goes...",1.278,102,0,2021-10-07,3.000,4,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, Français]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",21.801,96,15055,2023-03-24,6.474,38,"[Canada, United States of America]"
2,One True Loves,One True Loves,0,"[Romance, Comedy, Drama]",en,"[English, Español]",,Emma and Jesse are living the perfect life tog...,12.031,100,37820,2023-04-07,6.513,77,"[Czech Republic, United States of America]"
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,39.312,95,77145,2023-04-13,6.000,2,[United States of America]
4,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Español, Português]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.624,116,3395595,2023-03-31,6.871,89,[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Frantz,Frantz,0,"[History, Drama, Romance]",fr,"[Deutsch, Français]",,"In the aftermath of WWI, a young German who gr...",13.282,113,7478354,2016-09-07,7.365,671,"[France, Germany]"
173,The Other Half,The Other Half,0,"[Drama, Romance]",en,[English],http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,6.874,103,0,2016-12-02,6.300,24,[Canada]
174,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[Türkçe, English]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",13.046,111,413844,2017-03-10,6.164,247,"[Turkey, United States of America]"
175,Everybody Loves Somebody,Everybody Loves Somebody,2000000,"[Romance, Comedy]",en,"[English, Español]",,"On the surface, Clara Barron seems to have it ...",6.589,102,0,2017-02-10,6.538,159,[Mexico]


### Merge and Clean the Data for Export

In [36]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_movies_list_df, reviews_df_k, on='title', how="inner")
merged_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Español],,"Argentina, 1970s. A desperate young woman goes...",1.278,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, Français]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",21.801,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Comedy, Drama]",en,"[English, Español]",,Emma and Jesse are living the perfect life tog...,12.031,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,39.312,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Español, Português]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.624,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Frantz,Frantz,0,"[History, Drama, Romance]",fr,"[Deutsch, Français]",,"In the aftermath of WWI, a young German who gr...",13.282,113,...,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",None,None,The Wounds of War and Love,None,None,None,By Stephen Holden,"[{'firstname': 'Stephen', 'middlename': None, ...",None
166,The Other Half,The Other Half,0,"[Drama, Romance]",en,[English],http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,6.874,103,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
167,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[Türkçe, English]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",13.046,111,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
168,Everybody Loves Somebody,Everybody Loves Somebody,2000000,"[Romance, Comedy]",en,"[English, Español]",,"On the surface, Clara Barron seems to have it ...",6.589,102,...,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [38]:
merged_df.columns

Index(['title', 'original_title', 'budget', 'genre', 'language',
       'spoken_languages', 'homepage', 'overview', 'popularity', 'runtime',
       'revenue', 'release_date', 'vote_average', 'vote_count',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [40]:
merged_df['runtime'].head(2)

0    102
1     96
Name: runtime, dtype: int64

In [42]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_fixing_list = ['genre','spoken_languages','production_countries']

# Create a list of characters to remove
character_replace_list = ["[","]","'",]

def remove_characters(s):
    for char in character_replace_list:
        s = s.replace(char, "")
    return s

# Loop through the list of columns to fix
for col in columns_fixing_list:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
    merged_df[col] = merged_df[col].apply(remove_characters)

# Display the fixed DataFrame
merged_df[columns_fixing_list]

,genre,spoken_languages,production_countries
0,"Drama, Mystery, Thriller, Horror",Español,Argentina
1,"Drama, Romance","English, Français","Canada, United States of America"
2,"Romance, Comedy, Drama","English, Español","Czech Republic, United States of America"
3,Documentary,English,United States of America
4,"Drama, Crime","English, Español, Português",United States of America
...,...,...,...
165,"History, Drama, Romance","Deutsch, Français","France, Germany"
166,"Drama, Romance",English,Canada
167,"Romance, Drama, War","Türkçe, English","Turkey, United States of America"
168,"Romance, Comedy","English, Español",Mexico


In [44]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=['byline.person'])
merged_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Español,,"Argentina, 1970s. A desperate young woman goes...",1.278,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, Français",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",21.801,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Comedy, Drama",en,"English, Español",,Emma and Jesse are living the perfect life tog...,12.031,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,39.312,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Español, Português",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.624,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Frantz,Frantz,0,"History, Drama, Romance",fr,"Deutsch, Français",,"In the aftermath of WWI, a young German who gr...",13.282,113,...,708,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",None,None,The Wounds of War and Love,None,None,None,By Stephen Holden,None
166,The Other Half,The Other Half,0,"Drama, Romance",en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,6.874,103,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
167,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"Romance, Drama, War",en,"Türkçe, English",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",13.046,111,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None
168,Everybody Loves Somebody,Everybody Loves Somebody,2000000,"Romance, Comedy",en,"English, Español",,"On the surface, Clara Barron seems to have it ...",6.589,102,...,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,None


In [46]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

In [48]:
merged_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Español,,"Argentina, 1970s. A desperate young woman goes...",1.278,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"English, Français",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",21.801,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Comedy, Drama",en,"English, Español",,Emma and Jesse are living the perfect life tog...,12.031,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,English,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,39.312,95,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,"Drama, Crime",en,"English, Español, Português",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,9.624,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,Frantz,Frantz,0,"History, Drama, Romance",fr,"Deutsch, Français",,"In the aftermath of WWI, a young German who gr...",13.282,113,...,708,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",None,None,The Wounds of War and Love,None,None,None,By Stephen Holden,None
162,The Other Half,The Other Half,0,"Drama, Romance",en,English,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,6.874,103,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None
163,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"Romance, Drama, War",en,"Türkçe, English",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",13.046,111,...,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,None
164,Everybody Loves Somebody,Everybody Loves Somebody,2000000,"Romance, Comedy",en,"English, Español",,"On the surface, Clara Barron seems to have it ...",6.589,102,...,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,None


In [50]:
# Export data to CSV without the index
merged_df.to_csv('output/collected_data.csv', index=False)